In [232]:
"""
Скрипт для обработки данных полученных из яндекс формы по изучению возможностей ПОО к организации кружков для школьников
"""

'\nСкрипт для обработки данных полученных из яндекс формы по изучению возможностей ПОО к организации кружков для школьников\n'

In [233]:
import pandas as pd
import openpyxl

In [234]:
df = pd.read_excel('data/Мониторинг  ПОО кружки.xlsx')
df = df.iloc[:,2:] # отсекаем колонки со служебной информацией яндекса
df.fillna('Пусто',inplace=True) # заполняем пустоту
out_df = pd.DataFrame(columns=['ПОО','ФИО заполнившего','Должность','Телефон','Лицензия','Мастерская','Кружок','Количество рабочих мест','Условия для проведения','Кадровое обеспечение'])

In [235]:
df

,Выберите свое ПОО,ФИО заполняющего,Должность,Номер телефона заполняющего,"Укажите серию, номер, дату выдачи лицензии на реализацию программ дополнительного образования и срок ее действия",Название мастерской [1],"Наименование программ дополнительного образования для обучающихся общеобразовательных организаций, которые возможно проводить в данной мастерской [1]",Количество рабочих мест (сколько школьников могут заниматься одновременно) [1],Требуемые ресурсы для проведения [1],Название мастерской [2],...,Требуемые ресурсы для проведения [2],Название мастерской [3],"Наименование программ дополнительного образования для обучающихся общеобразовательных организаций, которые возможно проводить в данной мастерской [3]",Количество рабочих мест (сколько школьников могут заниматься одновременно) [3],Требуемые ресурсы для проведения [3],"Кого вы будете привлекать в качестве преподавателей программ дополнительного образования обучающихся общеобразовательных организаций. / Преподаватели/мастера/сотрудники, которые работают в ПОО",Кого вы будете привлекать в качестве преподавателей программ дополнительного образования обучающихся общеобразовательных организаций. / Студенты старших курсов ПОО,Кого вы будете привлекать в качестве преподавателей программ дополнительного образования обучающихся общеобразовательных организаций. / Студенты ВУЗов,"Кого вы будете привлекать в качестве преподавателей программ дополнительного образования обучающихся общеобразовательных организаций. / Наемные преподаватели (по договору ГПХ, например представители работодателя-партнера и т.д.)",Кого вы будете привлекать в качестве преподавателей программ дополнительного образования обучающихся общеобразовательных организаций. / Другое
0,Улан-Удэнский авиационный техникум (УУАТ),Иванов Иван Иванович,заместитель директора,8950315855,№ 14,IT,Юный программист,8,ничего,Пусто,...,Пусто,Пусто,Пусто,Пусто,Пусто,"Преподаватели/мастера/сотрудники, которые рабо...",Студенты старших курсов ПОО,Пусто,"Наемные преподаватели (по договору ГПХ, наприм...",Пусто
1,Бурятский республиканский информационно-эконом...,Лужие Павел Сихайлович,директор,89421245455,345345,Локомотив,Железнодорожное движение для школьников,15,"10 тысяч на оплату труда педагога, 25 метров к...",Слесарная,...,•\tЗачем это нужно\n•\tПоказать заполненные фа...,Ветеринария,Юный животновод,5.0,5 баранов,"Преподаватели/мастера/сотрудники, которые рабо...",Студенты старших курсов ПОО,Студенты ВУЗов,"Наемные преподаватели (по договору ГПХ, наприм...",Другое


In [236]:
border = 5 #  количество колонок с информацией о ПОО
offset = 4 # количество колонок которые занимают данные по мастерским
# максимальное количество мастерских
number_mastery = len([name_column for name_column in df.columns if 'Название мастерской' in name_column])
threshold =border + offset * number_mastery


In [237]:
df.iloc[:,border:threshold].columns

Index(['Название мастерской [1]',
       'Наименование программ дополнительного образования для обучающихся общеобразовательных организаций, которые возможно проводить в данной мастерской [1]',
       'Количество рабочих мест (сколько школьников могут заниматься одновременно) [1]',
       'Требуемые ресурсы для проведения [1]', 'Название мастерской [2]',
       'Наименование программ дополнительного образования для обучающихся общеобразовательных организаций, которые возможно проводить в данной мастерской [2]',
       'Количество рабочих мест (сколько школьников могут заниматься одновременно) [2]',
       'Требуемые ресурсы для проведения [2]', 'Название мастерской [3]',
       'Наименование программ дополнительного образования для обучающихся общеобразовательных организаций, которые возможно проводить в данной мастерской [3]',
       'Количество рабочих мест (сколько школьников могут заниматься одновременно) [3]',
       'Требуемые ресурсы для проведения [3]'],
      dtype='object')

In [238]:
for idx,row in enumerate(df.itertuples()):
    count = 6 # счетчик для отсчета обработанных срезов
    poo_data = list(row[1:count])
    print(poo_data)
    for mastery in range(number_mastery):
        data = list(row[count:count+offset])
        if data[0] !='Пусто':
            temp_lst = poo_data.copy()

            temp_lst.extend(data)
            
            # получаем строку с кадровым обеспечением
            temp_kadr_lst = [value for value in row[threshold+1:] if value != 'Пусто']
            temp_lst.append(';'.join(temp_kadr_lst))

            temp_row = pd.DataFrame(columns=['ПОО','ФИО заполнившего',
                                             'Должность','Телефон','Лицензия','Мастерская','Кружок','Количество рабочих мест',
                                             'Условия для проведения','Кадровое обеспечение'],
                                   data=[temp_lst])
            
            out_df = pd.concat([out_df,temp_row],ignore_index=True)
            
                                             
        
        
        
        count += offset



['Улан-Удэнский авиационный техникум (УУАТ)', 'Иванов Иван Иванович', 'заместитель директора', 8950315855, '№ 14']
['Бурятский республиканский информационно-экономический техникум (БРИЭТ)', 'Лужие  Павел Сихайлович', 'директор', 89421245455, '345345']


In [239]:
out_df

,ПОО,ФИО заполнившего,Должность,Телефон,Лицензия,Мастерская,Кружок,Количество рабочих мест,Условия для проведения,Кадровое обеспечение
0,Улан-Удэнский авиационный техникум (УУАТ),Иванов Иван Иванович,заместитель директора,8950315855,№ 14,IT,Юный программист,8,ничего,"Преподаватели/мастера/сотрудники, которые рабо..."
1,Бурятский республиканский информационно-эконом...,Лужие Павел Сихайлович,директор,89421245455,345345,Локомотив,Железнодорожное движение для школьников,15,"10 тысяч на оплату труда педагога, 25 метров к...","Преподаватели/мастера/сотрудники, которые рабо..."
2,Бурятский республиканский информационно-эконом...,Лужие Павел Сихайлович,директор,89421245455,345345,Слесарная,Юный слесарь,4.0,•\tЗачем это нужно\n•\tПоказать заполненные фа...,"Преподаватели/мастера/сотрудники, которые рабо..."
3,Бурятский республиканский информационно-эконом...,Лужие Павел Сихайлович,директор,89421245455,345345,Ветеринария,Юный животновод,5.0,5 баранов,"Преподаватели/мастера/сотрудники, которые рабо..."


In [240]:
kadr_df=out_df.drop_duplicates(subset=['ПОО']) # удаляем дубликаты чтобы подсчитать кадровое обеспечение
# создаем словарь
dct_kadr = {'Преподаватели/мастера/сотрудники, которые работают в ПОО':0,'Студенты старших курсов ПОО':0,
            'Студенты ВУЗов':0,'Наемные преподаватели (по договору ГПХ, например представители работодателя-партнера и т.д.)':0,'Другое':0,}

In [241]:
for row in kadr_df.itertuples():
    kadr_lst = row[-1].split(';')
    for kadr in kadr_lst:
        dct_kadr[kadr] += 1

In [242]:
dct_kadr

{'Преподаватели/мастера/сотрудники, которые работают в ПОО': 2,
 'Студенты старших курсов ПОО': 2,
 'Студенты ВУЗов': 1,
 'Наемные преподаватели (по договору ГПХ, например представители работодателя-партнера и т.д.)': 2,
 'Другое': 1}

In [243]:
out_kadr_df=pd.DataFrame(columns=['Категория','Количество'],data=dct_kadr.items())

In [244]:
out_kadr_df.sort_values(by=['Количество'],ascending=False,inplace=True)

In [245]:
count_poo_df =out_df.groupby(['ПОО']).agg({'Кружок':'count'}).reset_index()
count_poo_df.sort_values(by=['Кружок'],ascending=False,inplace=True)
count_krug_df =out_df.groupby(['ПОО','Мастерская']).agg({'Кружок':'count'}).reset_index()
count_krug_df.sort_values(by=['Кружок'],ascending=False,inplace=True)


In [246]:
with pd.ExcelWriter('data/Данные по мониторингу ПОО Доп образование.xlsx') as writer:
    out_df.to_excel(writer,sheet_name='Общие данные',index=0)
    out_kadr_df.to_excel(writer,sheet_name='Кадровое обеспечение',index=1)
    count_poo_df.to_excel(writer,sheet_name='Количество кружков в ПОО',index=2)
    count_krug_df.to_excel(writer,sheet_name='Количество кружков мастерские',index=3)
    

In [247]:
print('Lindy Booth')

Lindy Booth
